In [4]:
%config InlineBackend.figure_format = "retina"

import numpy as np
import matplotlib.pyplot as plt

from utils.parse_pars import parse_pars
from spamm.run_spamm import spamm
import run_fe, run_pl, run_bc, run_hg, run_el
from utils.add_in_quadrature import add_in_quadrature
from spamm.analysis import plot_best_models, plot_chains
from spamm.Samples import Samples
import corner

IndentationError: unexpected indent (run_el.py, line 30)

In [ ]:
def generate_data(components=None, comp_params=None):

    all_wls = []
    all_fluxes = []
    all_errs = []
    comb_p = {}
    comp_names = {}
    
    for component in components:
        component = component.upper()
        if component == "PL":
            comp_wl, comp_flux, comp_err, comp_p = run_pl.create_pl(comp_params["PL"])
            comp_names["PL"] = True

        elif component == "NEL":
            comp_wl, comp_flux, comp_err, comp_p = run_ne.create_ne(comp_params["NEL"])
            comp_names["NEL"] = True
            
        all_fluxes.append(comp_flux)
        all_wls.append(comp_wl)
        all_errs.append(comp_err)
        comb_p = {**comb_p, **comp_p}

    comb_wl = wl
    comb_flux = np.sum(all_fluxes, axis=0)
    comb_err = add_in_quadrature(all_errs)
    
    print(f"{LINEOUT}\nUsing components: {components}")
    
    return comb_wl, comb_flux, comb_err, all_fluxes, comp_names

In [ ]:
# This should be a wllength range from 1000-10,000A, every 0.5A
TEST_wl = parse_pars()["testing"]

# Create wllength array from min, max and step values
wl = np.arange(TEST_wl["wl_min"], TEST_wl["wl_max"], TEST_wl["wl_step"])
wl = np.arange(1000, 10000, 0.5)

# These values were picked by hand to provide the most realistic power law.
PL_PARAMS = {"norm_PL": 1e-14,
             "slope1": 2.3,
             "broken_pl": False,
             "wl": wl}

NEL_PARAMS = {"width": 50, 
              "amp_1": 10, 
              "center_1": 4830,
              "amp_2": 5, 
              "center_2": 6800,
              "amp_3": 8, 
              "center_3": 3000,
              "wl": wl}

LINEOUT = "#"*75

components=["PL", "NEL"]
comp_params={"PL": PL_PARAMS, "NEL": NEL_PARAMS}

_, flux, flux_err, all_fluxes, comp_names = generate_data(components=components, comp_params=comp_params)

noise = np.random.normal(0, 1, len(flux)) * 0.05*flux

fdeg = flux + noise

fig = plt.figure(figsize=(20,5))
plt.plot(wl, flux)
#plt.plot(wl, all_fluxes[0])
plt.title(f"Fake data spectrum")
plt.xlabel("wllength (Å)")
plt.ylabel("Flux");

In [ ]:
class EmissionLine():
    def __init__(self, name, wavelength, amplitude, width, num_comp):
        self.name = name
        self.wavelength = wavelength
        self.amplitude = amplitude
        self.width = width
        self.num_components = num_comp
        self.params = [param for i in range(num_comp) for param in (f"{name}_loc_{i+1}", f"{name}_amp_{i+1}")]

In [ ]:
test = EmissionLine("CIV", 1549.00, 10, 2, 1)
test.params

In [ ]:
spamm(complist=comp_names, inspectrum=(wl, flux, flux_err), comp_params=comp_params, n_walkers=150, n_iterations=1000, 
      outdir="combined_example", picklefile="combined", parallel=True, par_file="parameters.yaml")

In [ ]:
S = Samples("combined_example/combined.pickle.gz", outdir='combined_example', burn=600)
plot_best_models(S)

In [ ]:
fig, axes = plt.subplots(20, figsize=(10, 7), sharex=True)
samples = S.model.sampler.get_chain(discard=8000)
#tau = S.model.sampler.get_autocorr_time()
for i in range(20):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = S.model.sampler.get_chain(discard=6000, thin=10, flat=True)
fig = corner.corner(flat_samples);

In [ ]:
fig = plt.figure(figsize=(10, 7))
samples = S.model.sampler.get_log_prob()
plt.plot(samples[:,:]);
plt.xlim(30)
plt.ylim(-20000,0)

In [ ]:
plt.hist(samples, bins=200);
plt.xlim(-0.2e8, 0)

In [ ]:
import corner
flat_samples = S.model.sampler.get_chain(discard=1000, thin=20, flat=True)

fig = corner.corner(flat_samples, truths=[10, 2.3, 50, 10, 4830, 5, 6800, 8, 3000]);

In [ ]:
PL_PARAMS = {"norm_PL": 10,
             "slope1": 2.3,
             "broken_pl": False,
             "wl": wl}

# The normalizations are drawn from a gaussian sample with mu=9.06e-15,
# sigma=3.08946e-15 (from 0->template max flux). fe_width is halfway 
# between range in parameters. wl is very close to template span (1075-7535)
FE_PARAMS = {"fe_norm_1": 1.07988504e-14,
             "fe_norm_2": 6.91877436e-15,
             "fe_norm_3": 5e-15,# 8.68930476e-15, 
             "fe_width": 5450,
             "no_templates": 3,
             "wl": wl}

# These values are just the midpoints of the parameter space in parameters.yaml
BC_PARAMS = {"bc_norm": 3e-14,
             "bc_tauBE": 1.,
             "bc_logNe": 5.5,
             "bc_loffset": 0.,
             "bc_lwidth": 5050.,
             "bc_Te": 15250.,
             "bc_lines": 201.5,
             "wl": wl}

# These values are just the midpoints of the parameter space in parameters.yaml
HG_PARAMS = {"hg_norm_1": 4e-16,
             "hg_norm_2": 2e-16,
             "hg_stellar_disp": 515,
             "no_templates": 2,
             "wl": wl}

NEL_PARAMS = {"width": 50, 
              "amp_1": 10, 
              "center_1": 4830,
              "amp_2": 5, 
              "center_2": 6800,
              "amp_3": 8, 
              "center_3": 3000,
              "wl": wl}

LINEOUT = "#"*75

In [1]:
fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
samples = S.model.sampler.get_chain(discard=200)
#tau = S.model.sampler.get_autocorr_time()
for i in range(3):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");
axes[0].set_ylabel("amplitude")
axes[1].set_ylabel("alpha");

NameError: name 'plt' is not defined